In [2]:
# Base
import os
import pandas as pd
import numpy as np

from ml_logic.main import preprocess, preprocess_test

# AutoML with TPOT
import tpot

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
target_chosen = "nb_departures"
X_processed_df, y_processed_df, preprocessor, df_stations_reduced = preprocess(target_chosen)
X_test_processed, y_test_processed=preprocess_test(preprocessor, target_chosen, df_stations_reduced)


Raw data imported
Data cleaned and merged
features and target dataframes created
Geohash of stations computed


/home/anson/code/G-Dolle/DIVVY_BIKE/ml_logic/preprocessor.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_reduced.rename(columns={"name":"station_name"}, inplace=True)


Geohash of stations added to Features dataframe
features preprocessed
nb_departures picked as target
Preprocessing of Training set is done
Test Raw data imported
Test Data cleaned and merged
Test features and target dataframes created
nb_departures picked as target
Preprocessing of test set is done


In [6]:
X_processed_df.shape, X_test_processed.shape

((718651, 72), (539439, 72))

In [6]:
# generations=5, population_size=5
model = tpot.TPOTRegressor(generations=5, population_size=5, scoring='neg_mean_squared_error',
        verbosity=2, n_jobs=-1)
model.fit(X_processed_df, y_processed_df)
model.score(X_test_processed, y_test_processed)

                                                                                  
Generation 1 - Current best internal CV score: -14.374713240662174
                                                                                 
Generation 2 - Current best internal CV score: -14.374713240662174
                                                                               
Generation 3 - Current best internal CV score: -14.374713240662174
                                                                              
Generation 4 - Current best internal CV score: -14.374713240662174
                                                            
Generation 5 - Current best internal CV score: -14.194450421234865
                                                            
Best pipeline: ElasticNetCV(DecisionTreeRegressor(DecisionTreeRegressor(input_matrix, max_depth=9, min_samples_leaf=6, min_samples_split=13), max_depth=9, min_samples_leaf=6, min_samples_split=13), l1_ratio=0.1500000000

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


-9.017513880102376

In [7]:
# generations=10, population_size=5
model = tpot.TPOTRegressor(generations=10, population_size=5, scoring='neg_mean_squared_error',
        verbosity=2, n_jobs=-1)
model.fit(X_processed_df, y_processed_df)
model.score(X_test_processed, y_test_processed)

                                                                              
Generation 1 - Current best internal CV score: -15.653336254008853
                                                                            
Generation 2 - Current best internal CV score: -15.621119181161939
                                                                            
Generation 3 - Current best internal CV score: -15.621119181161939
                                                                            
Generation 4 - Current best internal CV score: -15.611652618164593
                                                                            
                                                                            
TPOT closed during evaluation in one generation.
                                                                            
                                                                            
TPOT closed prematurely. Will use the current best pipeline.
    

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


-8.621775379577924

In [ ]:
# generations=10, population_size=20
model = tpot.TPOTRegressor(generations=10, population_size=5, scoring='neg_mean_squared_error',
        verbosity=2, n_jobs=-1)
model.fit(X_processed_df, y_processed_df)
model.score(X_test_processed, y_test_processed)

                                                                              
Generation 1 - Current best internal CV score: -15.534825586181586
                                                                            
Generation 2 - Current best internal CV score: -15.534825586181586
                                                                            
Generation 3 - Current best internal CV score: -15.213895785064278
                                                                            
Generation 4 - Current best internal CV score: -15.213895785064278
                                                                            
Generation 5 - Current best internal CV score: -15.213895785064278
                                                                            
Generation 6 - Current best internal CV score: -14.984369466530989
                                                                            
Generation 7 - Current best internal CV score: -14.984369

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


-7.324031625413622

In [ ]:
# generations=20, population_size=20
model = tpot.TPOTRegressor(generations=10, population_size=5, scoring='neg_mean_squared_error',
        verbosity=2, n_jobs=-1)
model.fit(X_processed_df, y_processed_df)
model.score(X_test_processed, y_test_processed)

Optimization Progress:   0%|          | 0/110 [00:00<?, ?pipeline/s]

In [ ]:
# Export the result
model.export('tpot_departure.py')